# Causal Injection into a Neural Network

## Experiment on Boston Dataset

In [2]:
import numpy as np
np.set_printoptions(suppress=True)
import math
import networkx as nx
import random
import pandas as pd
#import tensorflow as tf
#Disable TensorFlow 2 behaviour
from sklearn.model_selection import KFold , train_test_split
from sklearn.preprocessing import StandardScaler  , MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import os
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
from CASTLE2 import CASTLE
from utils import gen_random_dag, gen_data_nonlinear, heat_mat, plot_DAG, DAG_retreive_np, save_pickle, load_pickle, random_stability
from signal import signal, SIGINT
from sys import exit
import argparse
import plotly.express as px
from pmlb import fetch_data


Instructions for updating:
non-resource variables are not supported in the long term


### Set parameters

In [3]:
## Parameters Definition
dag_type = 'boston' ##'fico','toy','random'
csv = None
# csv = './data/fico/WOE_Rud_data.csv'
# csv_y = './data/fico/y_data.csv'
# csv = 'synth_nonlinear.csv'
num_nodes = 20
branchf = 4
dataset_sz = 10000
output_log = f'castle_{dag_type}.log'
n_folds = 1
reg_lambda = 1
reg_beta = 5
gpu = ''
ckpt_file = os.path.join("./models/",dag_type,f'{dag_type}.ckpt')
extension = ''
seed = 0

random_stability(seed, verbose=True)

force_refit = True ## Refits the model if already present, False will only load it from ckpt_file
tuned = False ## Loads a tuned model saved in ckpt_file + "tuned"

tune = False ## Injects a loaded_adj into a model loaded from ckpt_file 
loaded_adj = None ## Adjacency matrix to be injected


Random seed 0 set for:
 - PYTHONHASHSEED (env)
 - random
 - NumPy
 - TensorFlow
 - PyTorch
   -> deterministic


### Generate or Load data

In [4]:
run=True
if run:
    scaler = StandardScaler()

    if dag_type == 'random' :
        def swap_cols(df, a, b):
            df = df.rename(columns = {a : 'temp'})
            df = df.rename(columns = {b : a})
            return df.rename(columns = {'temp' : b})
        def swap_nodes(G, a, b):
            newG = nx.relabel_nodes(G, {a : 'temp'})
            newG = nx.relabel_nodes(newG, {b : a})
            return nx.relabel_nodes(newG, {'temp' : b})
        
        #Random DAG
        num_edges = int(num_nodes*branchf)
        G = gen_random_dag(num_nodes, num_edges, seed)

        noise = random.uniform(0.3, 1.0)
        print("Setting noise to ", noise)
        
        df = gen_data_nonlinear(G, SIZE = dataset_sz, var = noise).iloc[:dataset_sz]
        df_test =  gen_data_nonlinear(G, SIZE = int(dataset_sz*0.25), var = noise)
        
        for i in range(len(G.edges())):
            if len(list(G.predecessors(i))) > 0:
                df = swap_cols(df, str(0), str(i))
                df_test = swap_cols(df_test, str(0), str(i))
                G = swap_nodes(G, 0, i)
                break     
                
        df = pd.DataFrame(df)
        df_test = pd.DataFrame(df_test)  
        
        #print("Number of parents of G", len(list(G.predecessors(i))))
        print("Edges = ", len(G.edges()), list(G.edges()))
        
    elif dag_type == 'toy':
        '''
        Toy DAG
        The node '0' is the target in the Toy DAG
        '''
        G = nx.DiGraph()
        for i in range(10):
            G.add_node(i)
        G.add_edge(1,2)
        G.add_edge(1,3)
        G.add_edge(1,4)
        G.add_edge(2,5)
        G.add_edge(2,0)
        G.add_edge(3,0)
        G.add_edge(3,6)
        G.add_edge(3,7)
        G.add_edge(6,9)
        G.add_edge(0,8)
        G.add_edge(0,9)

        if csv:
            df = pd.read_csv(csv)
            df_test = df.iloc[-1000:]
            df = df.iloc[:dataset_sz]
        else: 
            df = gen_data_nonlinear(G, SIZE = dataset_sz)
            df_test = gen_data_nonlinear(G, SIZE = 1000)

    elif dag_type == 'fico':
        df = pd.read_csv(csv)

        
    if dag_type == 'random':
        df = scaler.fit_transform(df)
    else:
        if csv:
            scaler.fit(pd.read_csv(csv))
            if dag_type != 'fico':
                df = scaler.transform(df)
        else:
            if dag_type not in ['adult','boston']:
                df = scaler.fit_transform(df)

    if dag_type in ['toy', 'random']:
        df_test = scaler.transform(df_test)
        
        X_test = df_test
        y_test = df_test[:,0]
        X_DAG = df

        # plot_DAG(G, G, 'py', debug=True)
        print(df.shape, df_test.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    elif dag_type == 'adult':
        # load dataset and descriptive statistics
        dataset = fetch_data(dag_type)

        print(dataset.head())    
        print(dataset.shape)
        print(dataset.columns)

        cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
                'marital-status', 'occupation', 'relationship', 'race', 'sex',
                'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

        print(dataset[cols].info())    
        print(dataset[cols].describe())
        print(dataset[cols].corr())    

        cols = list(dataset.columns)
        cols = [cols[-1]] + cols[:-1]
        dataset = dataset[cols]

        # load features and target from dataset
        X, y = fetch_data(dag_type, return_X_y=True)

        # Split Train and Test Datasets
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.25, shuffle = True, random_state = seed)  

        X_DAG = scaler.fit_transform(dataset)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        X_train = np.hstack((y_train[:,None],X_train))    
        X_test = np.hstack((y_test[:,None],X_test))    

        # plot_DAG(G, G, 'py', debug=True)
        print(X_DAG.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    elif dag_type == 'fico':
        label = 'RiskPerformance'
        y = pd.read_csv(os.path.join(csv_y))
        y = pd.get_dummies(y[label])[['Bad']].to_numpy()
        df.insert(loc=0, column=label, value=y)
        X_DAG = df.to_numpy()
        X_train, X_test, y_train, y_test = train_test_split(X_DAG, y, stratify = y, test_size = 0.25, shuffle = True, random_state = seed)
        print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    elif dag_type == 'boston':
        from sklearn.datasets import load_boston
        label = 'PRICE'

        df = pd.DataFrame(load_boston().data, columns=load_boston().feature_names)
        y = load_boston().target
        df.insert(loc=0, column=label, value=y)
        X_DAG = df.to_numpy()
        X_train, X_test, y_train, y_test = train_test_split(X_DAG, y, test_size = 0.2, shuffle = True, random_state = seed)
        print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(404, 14) (102, 14) (404,) (102,)


## Tau Optimisation

In [24]:
rerun_tau_optim = False
if rerun_tau_optim:
    %run -i main_fr.py --version="boston" ## set dataset_size to a number gt 506 eg [100000]

In [7]:
import re
from utils import load_pickle
import pandas as pd
import numpy as np

save = True
dset='100000'
version = 'boston'#'(adult2|adult)'
metric = 'MSE'
theta = '(([+-])?\d(\.\d+)?)'
folder = "./results/adjmats/"
list_edges = []
for filename in os.listdir(folder):
    match = re.search(f'W_est.{dset}.(\d).(\d).(.*?).{version}.pkl$', filename)
    if match:
        out_f, in_f, theta = match.groups()
        edges = sum(load_pickle(os.path.join(folder,filename), verbose=False).flatten()>0)
        list_edges.append((float(theta), out_f, in_f, edges))
n_edges = pd.DataFrame(list_edges).groupby(0, as_index=False).agg({3 : ['mean', 'std']})
n_edges.columns = n_edges.columns.droplevel()
n_edges.columns = ['theta','N_edges','Std']

folder = "./results/"
filename = f"Nested5FoldCASTLE.Reg.Synth.100000.{version}.pkl"
describe = load_pickle(os.path.join(folder,filename), verbose=False)
agg_stats = pd.DataFrame([(describe[c]['theta'],c[9],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({2 : ['count','max'], 3 : ['mean', 'std']})
agg_stats.columns = ['theta', 'count','folds',  'accuracy_mean', 'accuracy_std']
agg_stats=agg_stats.merge(n_edges,on='theta')
agg_stats= agg_stats.drop_duplicates(subset=['N_edges'])
sub = -0.2
agg_stats.loc[agg_stats['theta']==-1,'theta'] = sub

filter_theta = 0.8
filter_list = []
agg_stats = agg_stats[(agg_stats['theta']<=filter_theta) & (~round(agg_stats['theta'],3).isin(filter_list))]
display(agg_stats)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=agg_stats['theta'], y=agg_stats['accuracy_mean'], name=metric,         
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=agg_stats['accuracy_std'],
            visible=True)
            ),
    secondary_y=True, 
)

fig.add_trace(
    go.Scatter(x=agg_stats['theta'], y=agg_stats['N_edges'], name="Edges",
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=agg_stats['Std'],
            visible=True)
              ),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title='',
    legend={
        'y':-0.23,
        'x':0.85,
        'orientation':"h",
        'xanchor': 'center',
        'yanchor': 'bottom'},
    template='plotly_white',
    autosize=True,
    width=600, height=350, 
    margin=dict(
        l=10,
        r=10,
        b=0,
        t=10,
        pad=0
    ),
    font=dict(
        family='Serif',#"Courier New, monospace",
        size=18,
        # color="Black"
    )    
)

# Set x-axis title
fig.update_xaxes(showgrid=True,title={'text':r'$\text{Threshold } \tau$','font':{'size':18#, 'family': 'Courier New, monospace',
}},tickvals=list(np.round(np.arange(0, max(agg_stats['theta']), 0.5),1)),
)

# Set y-axes titles
fig.update_yaxes(title={'text':"Number of Edges"#,'font':{'size':18}
}, secondary_y=False)
fig.update_yaxes(showgrid=False,nticks=6,zeroline=False, title={'text':metric#,'font':{'size':18}
}, #nticks=13,
secondary_y=True)


fig.add_annotation(x=0.02, y=-0.094,
            text="CASTLE",
            showarrow=False,
            yshift=0,
            xref='paper',
            yref='paper',)
chosen_tau=0.13
y_co = agg_stats[agg_stats['theta']==chosen_tau]['accuracy_mean']
fig.add_annotation(x=chosen_tau, y=47.8,
            text=str(chosen_tau),
            showarrow=True,
            yshift=0,
            xref='x',ax=0,
            yref='y',ay=80)
fig.add_annotation(x=chosen_tau, y=20.363,
            text=str(20.4),
            showarrow=True,
            yshift=0,
            xref='x',ax=300,
            yref='y2',ay=0)
fig.add_annotation(x=chosen_tau, y=47.8,
            text=str(48),
            showarrow=True,
            yshift=0,
            xref='x',ax=-150,
            yref='y',ay=0)
            
fig.show()

output_folder = "figures"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

out_path = os.path.join(output_folder,f"plot_tauoptim_{version}.png")

if save:
    import kaleido
    fig.write_image(out_path)



<>:10: DeprecationWarning:

invalid escape sequence \d

<>:14: DeprecationWarning:

invalid escape sequence \d

<>:10: DeprecationWarning:

invalid escape sequence \d

<>:14: DeprecationWarning:

invalid escape sequence \d

<>:10: DeprecationWarning:

invalid escape sequence \d

<>:14: DeprecationWarning:

invalid escape sequence \d

<ipython-input-7-eced301e309a>:10: DeprecationWarning:

invalid escape sequence \d

<ipython-input-7-eced301e309a>:14: DeprecationWarning:

invalid escape sequence \d



,theta,count,folds,accuracy_mean,accuracy_std,N_edges,Std
0,-0.200,25,5,22.231392,7.387012,182.000000,0.000000
1,0.000,25,5,18.507690,7.042780,91.000000,0.000000
15,0.037,10,5,16.818329,6.783890,90.500000,0.527046
16,0.066,25,5,18.437362,6.868943,86.000000,1.290994
17,0.132,25,5,20.363651,5.233823,47.800000,7.942502
18,0.198,25,5,40.765085,27.205065,32.800000,8.098354
19,0.264,25,5,41.876700,27.307922,23.000000,6.390097
20,0.330,25,5,62.497449,29.583180,17.000000,5.916080
21,0.396,25,5,84.998406,12.277859,12.200000,5.416026
22,0.462,25,5,84.606197,12.862441,9.800000,4.804512


## Load Adjacency Matrix for chosen Tau

In [7]:
import re
dset='100000'
theta = '0.132'
version = 'boston'
folder = "./results/adjmats/"
list_edges = []
count = 0
W_est = np.zeros((df.shape[1],df.shape[1]))
for filename in os.listdir(folder):
#     print(filename)
    if version in ['v3','p1']:
        match = re.search(f'W_est.{dset}.([\w]+).([\w.]+).{version}.pkl$', filename)
    else:
        match = re.search(f'W_est.{dset}.(\d).(\d).{theta}(\d+)?.{version}.pkl$', filename)
    if match:
#         print( match.groups())
#         print(filename)
        out_f, in_f, theta = match.groups()
        w_est = load_pickle(os.path.join(folder,filename), verbose=False)
#         print(sum(w_est.flatten()>0))
#         print(w_est)
        W_est += w_est
#         print(W_est)
        count += 1
W_est = W_est/count
# print(W_est)
# print((W_est).flatten())
# print(sum((W_est).flatten()>0))
tau = 0.132
print("tau:",tau)
maxed_adj = DAG_retreive_np(W_est, tau)
heat_mat(maxed_adj, names= df.columns.values)
plot_DAG(maxed_adj, graphic_type = "py", names = list(df.columns))

tau: 0.132


,PRICE,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
PRICE,0.0,0.324,0.0,0.0,0.0,0.0,0.0,0.29,0.0,0.0,0.49,0.0,1.18,0.0
CRIM,0.0,0.0,1.03,0.0,0.0,0.0,0.0,0.657,0.0,0.432,1.44,0.0,1.43,0.0
ZN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.371,0.0,0.0,0.584,0.0,0.433,0.0
INDUS,0.0,0.0,0.767,0.0,0.0,0.0,0.0,0.592,0.0,0.0,0.664,0.0,0.437,0.0
CHAS,0.514,0.0,1.66,0.194,0.0,0.0,0.0,1.2,0.0,0.0,2.58,0.0,2.78,0.0
NOX,0.0,0.0,0.774,1.83,0.0,0.0,0.0,1.17,0.0,0.0,1.63,0.0,0.853,0.0
RM,0.355,0.0,0.0,0.0,0.0,0.0,0.0,0.67,0.0,0.0,0.74,0.0,0.839,0.309
AGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.363,0.0,0.376,0.0
DIS,0.277,0.535,0.932,0.0,0.0,0.0,0.0,1.02,0.0,0.0,1.33,0.0,1.15,0.0
RAD,0.0,0.0,0.456,0.0,0.0,0.0,0.0,0.392,0.0,0.0,1.69,0.0,0.0,0.0


Total Number of Edges in G: 51
Max in degree: 12
DAG: True


In [9]:
save_mat = False
if save_mat:
    pd.DataFrame(W_est).to_csv(f"{dag_type}_adj_matrix.csv", index=False)

## Dataset Size experiment

In [6]:
rerun_tau_optim = True
if rerun_tau_optim:
    # %run -i main_fr.py --version="boston_size2" --out_folds=1 --in_folds=10 --dataset_szs="50","100","200","300","400","506" --thetas='-1','0.132'
    %run -i main_fr.py --version="boston_size3" --out_folds=5 --in_folds=5 --dataset_szs="100","506" --thetas='-1','0.13'

Dset 100:   0%|          | 0/2 [00:00<?, ?it/s]

LR= 0.001
INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds100


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt


...

INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506
INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506_tuned


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506
INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506_tuned


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506
INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506_tuned


INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506
INFO:tensorflow:Restoring parameters from ./models/boston_size3/boston_size3.ckpt_ds506_tuned








Dset 506: 100%|██████████| 2/2 [30:48<00:00, 924.27s/it] 


In [12]:
debug=True

W_est = pd.read_csv(f"{dag_type}_adj_matrix.csv").to_numpy()

# v = f'{dag_type}_size' ##theta=0.06
v = f'{dag_type}_size2' ## theta=0.132

agg_stats_all = pd.DataFrame()

# size_list = [100,500,1000,4000,8000] ##fico
size_list = [50,100,200,300,400,506]
# out_folds = 5 ##fico
out_folds = 1
# metric = 'accuracy' ##fico
metric = 'MSE'
theta = 0.132 ## fico: 0.006

for size in size_list:
    # filename = f"5FoldCASTLE.Reg.Synth.{size}.pkl"
# for size in [50,100,200,500,1000]:
    filename = f"results/Nested{out_folds}FoldCASTLE.Reg.Synth.{size}.{v}.pkl"
    describe = load_pickle(os.path.join(filename), verbose=False)

    if debug:
        print(size)
        print(pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']}))

    agg_stats = pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']})
    agg_stats.columns = ['theta', 'folds', 'accuracy_mean', 'accuracy_std']

    agg_stats.loc[agg_stats['theta']==-1,'theta'] = -0.05
    agg_stats = agg_stats[agg_stats['theta']!= 0.05] 

    # display(agg_stats)

    n_edges = pd.DataFrame([(theta, nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False).number_of_edges()) for theta in agg_stats['theta']])

    n_edges.loc[n_edges[0]==-1,0] = -0.05
    n_edges.loc[n_edges[0]==-0.05,1] = nx.from_numpy_matrix(W_est, create_using=nx.DiGraph, parallel_edges=False).number_of_edges()

    # display(n_edges)

    n_edges_DAG = n_edges.merge(pd.DataFrame([(theta, nx.is_directed_acyclic_graph(nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False))) for theta in agg_stats['theta']]), on=0)

    n_edges_DAG.columns = ['theta', 'N_edges', 'is_DAG']

    agg_stats.merge(n_edges_DAG,on='theta')
    agg_stats['Dset_sz'] = size

    # display(agg_stats)

    agg_stats_all = pd.concat([agg_stats_all,agg_stats])

agg_stats_all

ds = agg_stats_all[['theta', 'accuracy_mean', 'accuracy_std','Dset_sz']]

ds['values'] = ["{0:.3f}".format(round(mean,3))+" ({0:.2f})".format(round(std,2)) for mean,std in zip(ds['accuracy_mean'],ds['accuracy_std']) ]
ds.loc[ds['theta']==-0.05,'theta'] = "CASTLE"
ds.loc[ds['theta']==theta,'theta'] = "Injected"
table = ds.pivot(index='Dset_sz',columns='theta', values='values')
table[["CASTLE","Injected"]]
display(table)


def compute_top_2(names):
    t = ''
    pcount = round(len(names)/len(np.unique(names)))
    _, idx = np.unique(names, return_index=True)
    names = names[np.sort(idx)]
    print(names)
    for i in range(len(names)):
        t += ' & \multicolumn{' + str(pcount) + '}{c}{' + str(names[i]) + '}'
    return t
            
# top1 = '\multirow{2}{*}{Dataset} & \multirow{2}{*}{BC$^{*}$}' + compute_top(table.columns.values[1:]) + '\\\\'
top1 = 'Dataset' + compute_top_2(table.columns.values) + '\\\\ \n\hline '

print_table = lambda table, top : table.to_latex(escape = False, index = True, index_names = False, header =False,
                     formatters=[(lambda x : '\!' + str(x) + '\!') for col in table.columns.values],
#                      header = [f'\!\!{nice_relations[col[-1:]] if col != "BC" else ""}\!\!' for col in table.columns.values],\
                     column_format = 'r' + 'c' * (len(table.columns.values)),
                    ).replace(r'\toprule', top).replace(r'\midrule', r'\hline').replace(r'\bottomrule', r'\hline').replace('.0%', '%').replace('\!%', '\!0%').replace('%', '\%')

print(print_table(table, top1))


# v = f'{dag_type}_size' ##theta=0.06
v = f'{dag_type}_size' ## theta=0.132

agg_stats_all = pd.DataFrame()

# size_list = [100,500,1000,4000,8000] ##fico
size_list = [50,100,200,300,400,506]
# out_folds = 5 ##fico
out_folds = 1
# metric = 'accuracy' ##fico
metric = 'MSE'
theta = 0.05 ## fico: 0.006

for size in size_list:
    # filename = f"5FoldCASTLE.Reg.Synth.{size}.pkl"
# for size in [50,100,200,500,1000]:
    filename = f"results/Nested{out_folds}FoldCASTLE.Reg.Synth.{size}.{v}.pkl"
    describe = load_pickle(os.path.join(filename), verbose=False)

    if debug:
        print(size)
        print(pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']}))

    agg_stats = pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']})
    agg_stats.columns = ['theta', 'folds', 'accuracy_mean', 'accuracy_std']

    agg_stats.loc[agg_stats['theta']==-1,'theta'] = -0.05

    # display(agg_stats)

    n_edges = pd.DataFrame([(theta, nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False).number_of_edges()) for theta in agg_stats['theta']])

    n_edges.loc[n_edges[0]==-1,0] = -0.05
    n_edges.loc[n_edges[0]==-0.05,1] = nx.from_numpy_matrix(W_est, create_using=nx.DiGraph, parallel_edges=False).number_of_edges()

    # display(n_edges)

    n_edges_DAG = n_edges.merge(pd.DataFrame([(theta, nx.is_directed_acyclic_graph(nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False))) for theta in agg_stats['theta']]), on=0)

    n_edges_DAG.columns = ['theta', 'N_edges', 'is_DAG']

    agg_stats.merge(n_edges_DAG,on='theta')
    agg_stats['Dset_sz'] = size

    # display(agg_stats)

    agg_stats_all = pd.concat([agg_stats_all,agg_stats])

agg_stats_all

ds = agg_stats_all[['theta', 'accuracy_mean', 'accuracy_std','Dset_sz']]

ds['values'] = ["{0:.3f}".format(round(mean,3))+" ({0:.2f})".format(round(std,2)) for mean,std in zip(ds['accuracy_mean'],ds['accuracy_std']) ]
ds.loc[ds['theta']==-0.05,'theta'] = "CASTLE"
ds.loc[ds['theta']==theta,'theta'] = "Injected"
table = ds.pivot(index='Dset_sz',columns='theta', values='values')
table[["CASTLE","Injected"]]
display(table)


def compute_top_2(names):
    t = ''
    pcount = round(len(names)/len(np.unique(names)))
    _, idx = np.unique(names, return_index=True)
    names = names[np.sort(idx)]
    print(names)
    for i in range(len(names)):
        t += ' & \multicolumn{' + str(pcount) + '}{c}{' + str(names[i]) + '}'
    return t
            
# top1 = '\multirow{2}{*}{Dataset} & \multirow{2}{*}{BC$^{*}$}' + compute_top(table.columns.values[1:]) + '\\\\'
top1 = 'Dataset' + compute_top_2(table.columns.values) + '\\\\ \n\hline '

print_table = lambda table, top : table.to_latex(escape = False, index = True, index_names = False, header =False,
                     formatters=[(lambda x : '\!' + str(x) + '\!') for col in table.columns.values],
#                      header = [f'\!\!{nice_relations[col[-1:]] if col != "BC" else ""}\!\!' for col in table.columns.values],\
                     column_format = 'r' + 'c' * (len(table.columns.values)),
                    ).replace(r'\toprule', top).replace(r'\midrule', r'\hline').replace(r'\bottomrule', r'\hline').replace('.0%', '%').replace('\!%', '\!0%').replace('%', '\%')

print(print_table(table, top1))

<>:73: DeprecationWarning:

invalid escape sequence \m

<>:77: DeprecationWarning:

invalid escape sequence \h

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \%

<>:155: DeprecationWarning:

invalid escape sequence \m

<>:159: DeprecationWarning:

invalid escape sequence \h

<>:162: DeprecationWarning:

invalid escape sequence \!

<>:162: DeprecationWarning:

invalid escape sequence \!

<>:165: DeprecationWarning:

invalid escape sequence \!

<>:165: DeprecationWarning:

invalid escape sequence \!

<>:165: DeprecationWarning:

invalid escape sequence \%

<>:73: DeprecationWarning:

invalid escape sequence \m

<>:77: DeprecationWarning:

invalid escape sequence \h

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:80: DeprecationWarning:

invalid escap

50
       0   1          2           
         max       mean        std
0 -1.000  10  85.483641  13.190258
1  0.050  10  45.906193   6.956744
2  0.132  10  88.201156  10.319166
100
       0   1          2          
         max       mean       std
0 -1.000  10  58.891078  9.370301
1  0.050  10  35.159560  3.663413
2  0.132  10  82.859168  1.946462
200
       0   1          2          
         max       mean       std
0 -1.000  10  41.068761  6.114584
1  0.132  10  30.805192  0.544166
300
       0   1          2          
         max       mean       std
0 -1.000  10  38.491383  5.177657
1  0.132  10  29.403418  0.987856
400
       0   1          2          
         max       mean       std
0 -1.000  10  32.798226  4.401397
1  0.132  10  28.861981  0.489881
506
       0   1          2          
         max       mean       std
0 -1.000  10  32.088942  5.314515
1  0.132  10  29.032117  1.156509


theta,CASTLE,Injected
Dset_sz,,
50,85.484 (13.19),88.201 (10.32)
100,58.891 (9.37),82.859 (1.95)
200,41.069 (6.11),30.805 (0.54)
300,38.491 (5.18),29.403 (0.99)
400,32.798 (4.40),28.862 (0.49)
506,32.089 (5.31),29.032 (1.16)


['CASTLE' 'Injected']
\begin{tabular}{rcc}
Dataset & \multicolumn{1}{c}{CASTLE} & \multicolumn{1}{c}{Injected}\\ 
\hline 
50  &  \!85.484 (13.19)\! &  \!88.201 (10.32)\! \\
100 &   \!58.891 (9.37)\! &   \!82.859 (1.95)\! \\
200 &   \!41.069 (6.11)\! &   \!30.805 (0.54)\! \\
300 &   \!38.491 (5.18)\! &   \!29.403 (0.99)\! \\
400 &   \!32.798 (4.40)\! &   \!28.862 (0.49)\! \\
506 &   \!32.089 (5.31)\! &   \!29.032 (1.16)\! \\
\hline
\end{tabular}

50
      0   1          2           
        max       mean        std
0 -1.00  10  85.483641  13.190258
1  0.05  10  45.906193   6.956744
100
      0   1          2          
        max       mean       std
0 -1.00  10  58.891078  9.370301
1  0.05  10  35.159560  3.663413
200
      0   1          2          
        max       mean       std
0 -1.00  10  41.068761  6.114584
1  0.05  10  31.774236  1.426251
300
      0   1          2          
        max       mean       std
0 -1.00  10  38.491383  5.177657
1  0.05  10  29.416145  2.280317
400

theta,CASTLE,Injected
Dset_sz,,
50,85.484 (13.19),45.906 (6.96)
100,58.891 (9.37),35.160 (3.66)
200,41.069 (6.11),31.774 (1.43)
300,38.491 (5.18),29.416 (2.28)
400,32.798 (4.40),28.798 (0.80)
506,31.918 (5.18),27.985 (1.19)


['CASTLE' 'Injected']
\begin{tabular}{rcc}
Dataset & \multicolumn{1}{c}{CASTLE} & \multicolumn{1}{c}{Injected}\\ 
\hline 
50  &  \!85.484 (13.19)\! &  \!45.906 (6.96)\! \\
100 &   \!58.891 (9.37)\! &  \!35.160 (3.66)\! \\
200 &   \!41.069 (6.11)\! &  \!31.774 (1.43)\! \\
300 &   \!38.491 (5.18)\! &  \!29.416 (2.28)\! \\
400 &   \!32.798 (4.40)\! &  \!28.798 (0.80)\! \\
506 &   \!31.918 (5.18)\! &  \!27.985 (1.19)\! \\
\hline
\end{tabular}



In [13]:
debug=True

W_est = pd.read_csv(f"{dag_type}_adj_matrix.csv").to_numpy()

# v = f'{dag_type}_size' ##theta=0.06
v = f'{dag_type}_size3' ## theta=0.132

agg_stats_all = pd.DataFrame()

# size_list = [100,500,1000,4000,8000] ##fico
size_list = [100,506]
# out_folds = 5 ##fico
out_folds = 5
# metric = 'accuracy' ##fico
metric = 'MSE'
theta = 0.13 ## fico: 0.006

for size in size_list:
    # filename = f"5FoldCASTLE.Reg.Synth.{size}.pkl"
# for size in [50,100,200,500,1000]:
    filename = f"results/Nested{out_folds}FoldCASTLE.Reg.Synth.{size}.{v}.pkl"
    describe = load_pickle(os.path.join(filename), verbose=False)

    if debug:
        print(size)
        print(pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']}))

    agg_stats = pd.DataFrame([(describe[c]['theta'],describe[c]['fold'],describe[c][metric]) for c in describe]).groupby(0, as_index=False).agg({1 : ['max'], 2 : ['mean', 'std']})
    agg_stats.columns = ['theta', 'folds', 'accuracy_mean', 'accuracy_std']

    agg_stats.loc[agg_stats['theta']==-1,'theta'] = -0.05
    agg_stats = agg_stats[agg_stats['theta']!= 0.05] 

    # display(agg_stats)

    n_edges = pd.DataFrame([(theta, nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False).number_of_edges()) for theta in agg_stats['theta']])

    n_edges.loc[n_edges[0]==-1,0] = -0.05
    n_edges.loc[n_edges[0]==-0.05,1] = nx.from_numpy_matrix(W_est, create_using=nx.DiGraph, parallel_edges=False).number_of_edges()

    # display(n_edges)

    n_edges_DAG = n_edges.merge(pd.DataFrame([(theta, nx.is_directed_acyclic_graph(nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False))) for theta in agg_stats['theta']]), on=0)

    n_edges_DAG.columns = ['theta', 'N_edges', 'is_DAG']

    agg_stats.merge(n_edges_DAG,on='theta')
    agg_stats['Dset_sz'] = size

    # display(agg_stats)

    agg_stats_all = pd.concat([agg_stats_all,agg_stats])

agg_stats_all

ds = agg_stats_all[['theta', 'accuracy_mean', 'accuracy_std','Dset_sz']]

ds['values'] = ["{0:.3f}".format(round(mean,3))+" ({0:.2f})".format(round(std,2)) for mean,std in zip(ds['accuracy_mean'],ds['accuracy_std']) ]
ds.loc[ds['theta']==-0.05,'theta'] = "CASTLE"
ds.loc[ds['theta']==theta,'theta'] = "Injected"
table = ds.pivot(index='Dset_sz',columns='theta', values='values')
table[["CASTLE","Injected"]]
display(table)


def compute_top_2(names):
    t = ''
    pcount = round(len(names)/len(np.unique(names)))
    _, idx = np.unique(names, return_index=True)
    names = names[np.sort(idx)]
    print(names)
    for i in range(len(names)):
        t += ' & \multicolumn{' + str(pcount) + '}{c}{' + str(names[i]) + '}'
    return t
            
# top1 = '\multirow{2}{*}{Dataset} & \multirow{2}{*}{BC$^{*}$}' + compute_top(table.columns.values[1:]) + '\\\\'
top1 = 'Dataset' + compute_top_2(table.columns.values) + '\\\\ \n\hline '

print_table = lambda table, top : table.to_latex(escape = False, index = True, index_names = False, header =False,
                     formatters=[(lambda x : '\!' + str(x) + '\!') for col in table.columns.values],
#                      header = [f'\!\!{nice_relations[col[-1:]] if col != "BC" else ""}\!\!' for col in table.columns.values],\
                     column_format = 'r' + 'c' * (len(table.columns.values)),
                    ).replace(r'\toprule', top).replace(r'\midrule', r'\hline').replace(r'\bottomrule', r'\hline').replace('.0%', '%').replace('\!%', '\!0%').replace('%', '\%')

print(print_table(table, top1))


100
      0   1           2           
        max        mean        std
0 -1.00   5  112.036437  91.056532
1  0.13   5   86.165875  13.753446
506
      0   1          2          
        max       mean       std
0 -1.00   5  21.954245  6.838826
1  0.13   5  20.455792  5.121598


<>:73: DeprecationWarning:

invalid escape sequence \m

<>:77: DeprecationWarning:

invalid escape sequence \h

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \%

<>:73: DeprecationWarning:

invalid escape sequence \m

<>:77: DeprecationWarning:

invalid escape sequence \h

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \!

<>:83: DeprecationWarning:

invalid escape sequence \%

<>:73: DeprecationWarning:

invalid escape sequence \m

<>:77: DeprecationWarning:

invalid escape sequence \h

<>:80: DeprecationWarning:

invalid escape sequence \!

<>:80: DeprecationWarning:

invalid escape seque

theta,CASTLE,Injected
Dset_sz,,
100,112.036 (91.06),86.166 (13.75)
506,21.954 (6.84),20.456 (5.12)


['CASTLE' 'Injected']
\begin{tabular}{rcc}
Dataset & \multicolumn{1}{c}{CASTLE} & \multicolumn{1}{c}{Injected}\\ 
\hline 
100 &  \!112.036 (91.06)\! &  \!86.166 (13.75)\! \\
506 &    \!21.954 (6.84)\! &   \!20.456 (5.12)\! \\
\hline
\end{tabular}



## Castle Fitting - with or without injection

In [ ]:
run=True ## to run the below set to True
n_folds=1
output_log = f'castle_{dag_type}_full.log'
n_folds = 1
ckpt_file = os.path.join("./models/",dag_type,f'{dag_type}_full.ckpt')
seed = 0

In [ ]:
if run:
        if n_folds > 1:
                kf = KFold(n_splits = n_folds, random_state = 1, shuffle = True)

                fold = 0
                REG_castle = []
                print("Dataset limits are", np.ptp(X_DAG), np.ptp(X_test), np.ptp(y_test))
                for train_idx, val_idx in kf.split(X_DAG):
                        fold += 1
                        print("fold = ", fold)
                        print("******* Doing dataset size = ", len(train_idx) , "****************")
                        X_train = X_DAG[train_idx]
                        y_train = np.expand_dims(X_DAG[train_idx][:,0], -1)
                        X_val = X_DAG[val_idx]
                        y_val = np.expand_dims(X_DAG[val_idx][:,0], -1)

                        w_threshold = 0.3
                        castle = CASTLE(num_train = X_DAG.shape[0], num_inputs = X_DAG.shape[1], reg_lambda = reg_lambda, reg_beta = reg_beta,
                                        w_threshold = w_threshold, ckpt_file = ckpt_file
                                        )

                        num_nodes = np.shape(X_DAG)[1]
                        castle.fit(X=X_train, y=y_train, num_nodes=num_nodes, X_val=X_val, y_val=y_val,
                                overwrite=True, tune=False, maxed_adj=None, tuned=False)

                        W_est = castle.pred_W(X_DAG, np.expand_dims(X_DAG[:,0], -1))
                        heat_mat(W_est)
                        
                        REG_castle.append(mean_squared_error(castle.pred(X_test), y_test))
                        print("MSE = ", mean_squared_error(castle.pred(X_test), y_test))

                        if fold > 1:
                        print("MEAN =", np.mean(REG_castle), "STD =", np.std(REG_castle))
        else:
                print("******* Doing dataset size = ", X_train.shape[0] , "****************")
                y_train = np.expand_dims(X_train[:,0], -1)
                X_val = X_test
                y_val = y_test

                w_threshold = 0.3
                hidden = int(X_DAG.shape[1]*3.2)
                
                castle = CASTLE(num_train = X_train.shape[0], num_inputs = X_train.shape[1],  n_hidden=hidden,
                                reg_lambda = reg_lambda, reg_beta = reg_beta, type_output='reg',
                                w_threshold = w_threshold, ckpt_file = ckpt_file, seed = seed
                                )
                castle.fit(X=X_train, y=y_train, num_nodes=X_train.shape[1], X_val=X_val, y_val=y_val,
                        overwrite=force_refit, tune=tune, tuned=tuned, maxed_adj=loaded_adj, seed = seed )
                print("MSE = ", mean_squared_error(castle.pred(X_test), y_test))

                W_est = castle.pred_W(X_DAG, np.expand_dims(X_DAG[:,0], -1))
                heat_mat(W_est)

******* Doing dataset size =  36631 ****************
Step 1, Loss= 3.0223 h_value: 0.0027160645
Val Loss= 0.1270
Step 2, Loss= 0.1298 h_value: 0.04360199
Val Loss= 0.1175
Step 3, Loss= 0.1204 h_value: 0.06536102
Val Loss= 0.1138
Step 4, Loss= 0.1159 h_value: 0.065068245
Val Loss= 0.1137
Step 5, Loss= 0.1159 h_value: 0.053347588
Val Loss= 0.1163
Step 6, Loss= 0.1200 h_value: 0.04725361
Val Loss= 0.1176
Step 7, Loss= 0.1205 h_value: 0.039466858
Val Loss= 0.1231
Step 8, Loss= 0.1248 h_value: 0.036218643
Val Loss= 0.1098
Step 9, Loss= 0.1131 h_value: 0.031040192
Val Loss= 0.1149
Step 10, Loss= 0.1178 h_value: 0.028759956
Val Loss= 0.1168
Step 11, Loss= 0.1190 h_value: 0.027578354
Val Loss= 0.1105
Step 12, Loss= 0.1129 h_value: 0.02609539
Val Loss= 0.1163
Step 13, Loss= 0.1185 h_value: 0.024947166
Val Loss= 0.1246
Step 14, Loss= 0.1265 h_value: 0.021176338
Val Loss= 0.1121
Step 15, Loss= 0.1143 h_value: 0.024569511
Val Loss= 0.1171
Step 16, Loss= 0.1205 h_value: 0.019417763
Val Loss= 0.1134

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.024,0.009,0.004,0.0,0.009,0.009,0.007,0.006,0.008,0.005,0.068,0.025,0.023,0.006
1,0.006,0.0,0.0,0.017,0.0,0.007,0.108,0.002,0.02,0.008,0.008,0.009,0.005,0.127,0.018
2,0.001,0.013,0.0,0.003,0.0,0.006,0.009,0.064,0.004,0.012,0.013,0.007,0.008,0.016,0.007
3,0.002,0.014,0.001,0.0,0.0,0.011,0.014,0.001,0.011,0.018,0.007,0.003,0.007,0.004,0.009
4,0.002,0.006,0.001,0.008,0.0,0.43,0.003,0.006,0.006,0.004,0.013,0.013,0.002,0.005,0.01
5,0.016,0.021,0.004,0.006,0.147,0.0,0.016,0.028,0.015,0.039,0.027,0.035,0.013,0.011,0.061
6,0.04,0.114,0.001,0.003,0.0,0.006,0.0,0.002,0.092,0.009,0.077,0.011,0.002,0.008,0.003
7,0.002,0.004,0.194,0.004,0.0,0.007,0.004,0.0,0.005,0.001,0.01,0.004,0.009,0.01,0.004
8,0.001,0.068,0.001,0.005,0.0,0.01,0.134,0.007,0.0,0.024,0.078,0.002,0.0,0.02,0.005
9,0.003,0.019,0.004,0.057,0.0,0.009,0.022,0.004,0.008,0.0,0.007,0.001,0.001,0.007,0.09


In [ ]:
if run:
    fig = px.histogram(W_est.flatten(), marginal="box")
    fig.show()

In [ ]:
if run:
    df = dataset
    heat_mat(W_est, names= df.columns.values)
    plot_DAG(W_est, graphic_type = "py", names = list(df.columns))

    tau = 0.029
    print("tau:",tau)
    maxed_adj = DAG_retreive_np(W_est, tau)
    heat_mat(maxed_adj, names= df.columns.values)
    plot_DAG(maxed_adj, graphic_type = "py", names = [a.split('_')[0] for a in df.columns.values])
    # plot_DAG(W_est, graphic_type = "py", names = [a.split('_')[0] for a in df.columns.values])


,target,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
target,0.0,0.024,0.009,0.004,0.0,0.009,0.009,0.007,0.006,0.008,0.005,0.068,0.025,0.023,0.006
age,0.006,0.0,0.0,0.017,0.0,0.007,0.108,0.002,0.02,0.008,0.008,0.009,0.005,0.127,0.018
workclass,0.001,0.013,0.0,0.003,0.0,0.006,0.009,0.064,0.004,0.012,0.013,0.007,0.008,0.016,0.007
fnlwgt,0.002,0.014,0.001,0.0,0.0,0.011,0.014,0.001,0.011,0.018,0.007,0.003,0.007,0.004,0.009
education,0.002,0.006,0.001,0.008,0.0,0.43,0.003,0.006,0.006,0.004,0.013,0.013,0.002,0.005,0.01
education-num,0.016,0.021,0.004,0.006,0.147,0.0,0.016,0.028,0.015,0.039,0.027,0.035,0.013,0.011,0.061
marital-status,0.04,0.114,0.001,0.003,0.0,0.006,0.0,0.002,0.092,0.009,0.077,0.011,0.002,0.008,0.003
occupation,0.002,0.004,0.194,0.004,0.0,0.007,0.004,0.0,0.005,0.001,0.01,0.004,0.009,0.01,0.004
relationship,0.001,0.068,0.001,0.005,0.0,0.01,0.134,0.007,0.0,0.024,0.078,0.002,0.0,0.02,0.005
race,0.003,0.019,0.004,0.057,0.0,0.009,0.022,0.004,0.008,0.0,0.007,0.001,0.001,0.007,0.09


Total Number of Edges in G: 210
Max in degree: 14
DAG: False
tau: 0.029


,target,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
target,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068,0.0,0.0,0.0
age,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127,0.0
workclass,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fnlwgt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
education,0.0,0.0,0.0,0.0,0.0,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
education-num,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039,0.0,0.035,0.0,0.0,0.061
marital-status,0.04,0.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077,0.0,0.0,0.0,0.0
occupation,0.0,0.0,0.194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
relationship,0.0,0.068,0.0,0.0,0.0,0.0,0.134,0.0,0.0,0.0,0.078,0.0,0.0,0.0,0.0
race,0.0,0.0,0.0,0.057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Total Number of Edges in G: 18
Max in degree: 3
DAG: True
